In [ ]:
#Входные данные это 3 файла .csv: выгрузка по абонентам населенного пункта, закрытия объемов за 2022 год и за 2023 год.
#Необходимо получить список лицевых счетов абонентов с наибольшей вероятностью нарушений.

In [ ]:
#импортируем библиотеки и производим чтение csv файлов

In [52]:
import pandas as pd
import sqlite3
pos = pd.read_csv('/Users/rinathamitov/Downloads/krs/poselok.csv')
g22 = pd.read_csv('/Users/rinathamitov/Downloads/krs/gaz22.csv')
g23 = pd.read_csv('/Users/rinathamitov/Downloads/krs/gaz23.csv')

In [53]:
#переименовываем для удобства столбцы, которые будем использовать для таблицы pos
#удаляем столбцы, которые не будем использовать
#приводим площадь отопливаемых помещений к нормальному виду

In [54]:
pos = pos.set_axis(['locality', 'ls', 'del1', 'del2', 'del3', 'street', 'house', 'del4', 'del5', 'del6', 'date_last_pr', 'del7', 
                  'del8', 'meter_verification', 'pr_pok', 'tek_pok', 'area', 'del9', 'del10', 'del11', 'del12'], axis=1)

pos.drop(['del1', 'del2', 'del3', 'del4', 'del5', 'del6', 'del7', 'del8', 'del9', 'del10', 'del11', 'del12'], axis=1, inplace=True)

pos['area'] = pos['area'].astype(str).str.replace(',', '.')  
pos['area'] = pd.to_numeric(pos['area'], errors='coerce')

In [55]:
#переименовываем для удобства столбцы, которые будем использовать для таблицы g22
#приводим закрытые объемы по месяцам за 2022 год к нормальному виду
#добавляем стобец, в котором вычисляется сумма закрытых объемов в период с января по сентябрь 2022 года
#удаляем столбцы, которые не будем использовать далее

In [56]:
g22 = g22.set_axis(['ls', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], axis=1)

for column in g22.columns[1:]:
    g22[column] = g22[column].astype(str).str.replace(',', '.')  
    g22[column] = pd.to_numeric(g22[column], errors='coerce') 
    
jan_sep_t = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep']
g22['jan_sep22'] = g22[jan_sep_t].sum(axis=1)

g22.drop(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov'], axis=1, inplace=True)

In [57]:
#переименовываем для удобства столбцы, которые будем использовать для таблицы g23
#приводим закрытые объемы по месяцам за 2023 год к нормальному виду
#добавляем стобец, в котором вычисляется сумма закрытых объемов в период с января по сентябрь 2023 года
#удаляем столбцы, которые не будем использовать далее

In [58]:
g23 = g23.set_axis(['ls', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep'], axis=1)

for column in g23.columns[1:]:
    g23[column] = g23[column].astype(str).str.replace(',', '.')  
    g23[column] = pd.to_numeric(g23[column], errors='coerce') 
    
g23['jan_sep23'] = g23[jan_sep_t].sum(axis=1)

g23.drop(['Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep'], axis=1, inplace=True)

In [59]:
#соединяем все три полученные таблицы по лицевому счету

In [60]:
total = pos.merge(g22, on='ls', how='left') \
           .merge(g23, on='ls', how='left')

In [61]:
#5 критериев оценки, выгрузка на сентябрь 2023, анализ будет на 9 октября 2023 года:
#1 - истек МПИ (0.44)
#2 - аномально низкое потребление в отопительный период (0.2)
#3 - снижение потребления относительно аналогичного периода за прошлый год больше чем на 30% (0.15)
#4 - возможность недопереданного объема (0.1)
#5 - длительное отсутствие проверок (0.1)
#пишем sql запрос, в котором выводим лицевой счет, улицу, дом, критерии
#записываем результаты в новую таблицу total_new

In [62]:
conn = sqlite3.connect('my_database.db')
total.to_sql('total', conn, if_exists='replace', index=False)
total_new = pd.read_sql_query("""
    SELECT ls, street, house,
        CASE WHEN meter_verification < '2023-10-09' THEN 0.44 ELSE 0 END as k1,
        CASE WHEN (Dec + Jan + Feb) < (area * 0.0108) THEN 0.2 ELSE 0 END as k2,
        CASE WHEN (jan_sep23 / jan_sep22) < 0.7 THEN 0.15 ELSE 0 END as k3,
        CASE WHEN pr_pok = tek_pok THEN 0.1 ELSE 0 END as k4,
        CASE WHEN julianday('2023-10-09') - julianday(date_last_pr) > 60 THEN 0.1 ELSE 0 END as k5
    FROM total
""", conn)
total_new.to_sql('total_new', conn, if_exists='replace', index=False)

186

In [63]:
#пишем sql запрос, в котором получаем 10 лицевых счетов абонентов с наибольшей вероятность нарушений

In [64]:
result = pd.read_sql_query("""
    SELECT ls as 'Лицевой счет', street as 'Улица', house as 'Дом', (k1 + k2 + k3 + k4 + k5) * 100 as 'Вероятность выявления нарушения, %'
    FROM total_new
    ORDER BY 4 desc
    LIMIT 10
""", conn)
print(result.to_string(index=False))

 Лицевой счет           Улица    Дом  Вероятность выявления нарушения, %
            2      пер Дружбы   д.23                                99.0
           89         ул Мира  д.231                                99.0
          101    ул Некрасова  д.124                                99.0
          119       ул Победы   д.23                                99.0
          148     ул Симонова   д.48                                84.0
           70 ул Крестьянская   д.25                                79.0
            1      пер Дружбы   д.21                                55.0
            3      пер Дружбы    д.6                                55.0
            4     пер Зеленый   д.6А                                55.0
            6  пер Котовского   д.6А                                55.0
